In [6]:
import requests
import pandas as pd
from pprint import pprint

CLIENT_ID = 'b4d023fcb19148bb998d675f89b6c302'
CLIENT_SECRET = '81930c900ac44ddcbbf9517abc26f15b'

In [2]:
AUTH_URL = 'https://accounts.spotify.com/api/token'

# POST
auth_response = requests.post(AUTH_URL, {
    'grant_type': 'client_credentials',
    'client_id': CLIENT_ID,
    'client_secret': CLIENT_SECRET,
})

# convert the response to JSON
auth_response_data = auth_response.json()

# save the access token
access_token = auth_response_data['access_token']

In [3]:
headers = {
    'Authorization': 'Bearer {token}'.format(token=access_token)
}

In [4]:
# base URL of all Spotify API endpoints
Search_URL = 'https://api.spotify.com/v1/search'

In [7]:
genre_artist_df = pd.read_csv('../Music-Research-Project-1/Resources/genre_artist_list.csv')
genre_artist_df.head()

,Genre,Genre ID,Artist
0,Pop,g.115,Michael Jackson
1,Pop,g.115,Madonna
2,Pop,g.115,Bruno Mars
3,Pop,g.115,Elton John
4,Pop,g.115,Taylor Swift


In [8]:
artist_name= "Linkin Park"

In [9]:
artist_info = requests.get(
Search_URL,
headers=headers,
params={'q': artist_name, 'type': 'artist'}).json()


In [11]:
uri = artist_info['artists']['items'][0]['id']
print(uri)

6XyY86QOPPrYVGvF9ch6wz


In [14]:
id = []

for artist in genre_artist_df['Artist']:
    try:
        artist_info = requests.get(
        Search_URL,
        headers=headers,
        params={'q': artist, 'type': 'artist'}).json()

        uri = artist_info['artists']['items'][0]['id']
        id.append(uri)
    except:
        print(f'{artist} not found on Spotify!')
        id.append('NaN')

Diplo feat. Lil Pump, Juicy J, Famous Dex & French Montana not found on Spotify!


In [15]:
len(id)

460

In [16]:
genre_artist_df['Spotify ID'] = id
genre_artist_df.head()

,Genre,Genre ID,Artist,Spotify ID
0,Pop,g.115,Michael Jackson,3fMbdgg4jU18AjLCKBhRSm
1,Pop,g.115,Madonna,6tbjWDEIzxoDsBA1FuhfPW
2,Pop,g.115,Bruno Mars,0du5cEVh5yTK9QJze8zA0C
3,Pop,g.115,Elton John,3PhoLpVuITZKcymswpck5b
4,Pop,g.115,Taylor Swift,06HL4z0CvFAxyc27GXpf02


In [17]:
mj_id = 'spotify:artist:3fMbdgg4jU18AjLCKBhRSm'

In [26]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth

sp = spotipy.Spotify(auth_manager=SpotifyOAuth(client_id=CLIENT_ID,
                                               client_secret=CLIENT_SECRET,
                                               redirect_uri='http://localhost:1234',
                                               scope="user-library-read"))


In [ ]:
tracks = sp.artist_top_tracks(mj_id,country='US')

In [45]:
print(tracks['tracks'][0]['album']['artists'][0]['name'])

Michael Jackson


In [ ]:
for track in tracks['tracks'][:10]:
    print('track    : ' + track['name'] + '| uri: ' + track['uri'])

In [36]:
track1 = sp.audio_features(
    'spotify:track:2bCQHF9gdG5BNDVuEIEnNk'
)
print(track1)

[{'danceability': 0.853, 'energy': 0.981, 'key': 8, 'loudness': -3.947, 'mode': 1, 'speechiness': 0.0751, 'acousticness': 0.247, 'instrumentalness': 0.468, 'liveness': 0.306, 'valence': 0.595, 'tempo': 118.193, 'type': 'audio_features', 'id': '2bCQHF9gdG5BNDVuEIEnNk', 'uri': 'spotify:track:2bCQHF9gdG5BNDVuEIEnNk', 'track_href': 'https://api.spotify.com/v1/tracks/2bCQHF9gdG5BNDVuEIEnNk', 'analysis_url': 'https://api.spotify.com/v1/audio-analysis/2bCQHF9gdG5BNDVuEIEnNk', 'duration_ms': 257760, 'time_signature': 4}]


In [40]:
type(track1[0])

dict

In [48]:
genre_artist_df[genre_artist_df['Artist'] == 'Diplo feat. Lil Pump, Juicy J, Famous Dex & French Montana'].index

Int64Index([383], dtype='int64')

In [49]:
cleaned_artist_df =genre_artist_df.drop(genre_artist_df.index[383], axis=0)

In [50]:
len(cleaned_artist_df)

459

In [56]:
print(genre_artist_df['Artist'].iloc[383])

Diplo feat. Lil Pump, Juicy J, Famous Dex & French Montana


In [58]:
error_dict = {'danceability': 'NA', 'energy': 'NA', 'key': 'NA', 'loudness': 'NA', 'mode': 'NA',
 'speechiness': 'NA', 'acousticness': 'NA', 'instrumentalness': 'NA', 'liveness': 'NA', 
 'valence': 'NA', 'tempo': 'NA', 'type': 'audio_features', 'id': 'NA', 
 'uri': 'NA', 'track_href': 'NA', 
 'analysis_url': 'NA', 'duration_ms': 'NA', 'time_signature': 'NA'}

In [61]:
track_features = []

for s_id in cleaned_artist_df['Spotify ID']:
    current_id = f'spotify:artist:{s_id}'
    current_tracks = sp.artist_top_tracks(current_id,country='US')
    try:
        for track in current_tracks['tracks'][:10]:
            track_info = {'Artist': track['album']['artists'][0]['name'], 'Track': track['name'], 'Track ID': track['uri']}
            track_audiofeat = sp.audio_features(track['uri'])
            track_info.update(track_audiofeat[0])
            track_features.append(track_info)
    except:
        name = track['name']
        print(f'{name} encountered an error!')
        track_info = {'Artist': track['album']['artists'][0]['name'], 'Track': track['name'], 'Track ID': track['uri']}
        track_info.update(error_dict)
        track_features.append(track_info)
    

50 Reggae Lovers Rock Songs - Continuous Mix encountered an error!


In [62]:
track_feat_df = pd.DataFrame(track_features)
track_feat_df.head()

,Artist,Track,Track ID,danceability,energy,key,loudness,mode,speechiness,acousticness,...,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Michael Jackson,Billie Jean,spotify:track:5ChkMS8OtdzJeqyybCc9R5,0.92,0.654,11,-3.051,0,0.0401,0.0236,...,0.036,0.847,117.046,audio_features,5ChkMS8OtdzJeqyybCc9R5,spotify:track:5ChkMS8OtdzJeqyybCc9R5,https://api.spotify.com/v1/tracks/5ChkMS8OtdzJ...,https://api.spotify.com/v1/audio-analysis/5Chk...,293827,4
1,Michael Jackson,Beat It,spotify:track:1OOtq8tRnDM8kG2gqUPjAj,0.776,0.867,3,-3.704,0,0.0479,0.0495,...,0.197,0.918,138.827,audio_features,1OOtq8tRnDM8kG2gqUPjAj,spotify:track:1OOtq8tRnDM8kG2gqUPjAj,https://api.spotify.com/v1/tracks/1OOtq8tRnDM8...,https://api.spotify.com/v1/audio-analysis/1OOt...,258040,4
2,Michael Jackson,Chicago,spotify:track:5BKKy9fIJL5uM9fz1SnqyP,0.608,0.756,7,-2.675,0,0.176,0.0784,...,0.187,0.55,176.086,audio_features,5BKKy9fIJL5uM9fz1SnqyP,spotify:track:5BKKy9fIJL5uM9fz1SnqyP,https://api.spotify.com/v1/tracks/5BKKy9fIJL5u...,https://api.spotify.com/v1/audio-analysis/5BKK...,245507,4
3,Michael Jackson,Smooth Criminal - 2012 Remaster,spotify:track:2bCQHF9gdG5BNDVuEIEnNk,0.853,0.981,8,-3.947,1,0.0751,0.247,...,0.306,0.595,118.193,audio_features,2bCQHF9gdG5BNDVuEIEnNk,spotify:track:2bCQHF9gdG5BNDVuEIEnNk,https://api.spotify.com/v1/tracks/2bCQHF9gdG5B...,https://api.spotify.com/v1/audio-analysis/2bCQ...,257760,4
4,Michael Jackson,Don't Stop 'Til You Get Enough,spotify:track:46eu3SBuFCXWsPT39Yg3tJ,0.878,0.821,11,-9.875,1,0.089,0.126,...,0.183,0.947,118.881,audio_features,46eu3SBuFCXWsPT39Yg3tJ,spotify:track:46eu3SBuFCXWsPT39Yg3tJ,https://api.spotify.com/v1/tracks/46eu3SBuFCXW...,https://api.spotify.com/v1/audio-analysis/46eu...,365467,4


In [63]:
merged_genre_tracks = pd.merge(track_feat_df,genre_artist_df,how='left', on='Artist')
merged_genre_tracks.head()

,Artist,Track,Track ID,danceability,energy,key,loudness,mode,speechiness,acousticness,...,type,id,uri,track_href,analysis_url,duration_ms,time_signature,Genre,Genre ID,Spotify ID
0,Michael Jackson,Billie Jean,spotify:track:5ChkMS8OtdzJeqyybCc9R5,0.92,0.654,11,-3.051,0,0.0401,0.0236,...,audio_features,5ChkMS8OtdzJeqyybCc9R5,spotify:track:5ChkMS8OtdzJeqyybCc9R5,https://api.spotify.com/v1/tracks/5ChkMS8OtdzJ...,https://api.spotify.com/v1/audio-analysis/5Chk...,293827,4,Pop,g.115,3fMbdgg4jU18AjLCKBhRSm
1,Michael Jackson,Beat It,spotify:track:1OOtq8tRnDM8kG2gqUPjAj,0.776,0.867,3,-3.704,0,0.0479,0.0495,...,audio_features,1OOtq8tRnDM8kG2gqUPjAj,spotify:track:1OOtq8tRnDM8kG2gqUPjAj,https://api.spotify.com/v1/tracks/1OOtq8tRnDM8...,https://api.spotify.com/v1/audio-analysis/1OOt...,258040,4,Pop,g.115,3fMbdgg4jU18AjLCKBhRSm
2,Michael Jackson,Chicago,spotify:track:5BKKy9fIJL5uM9fz1SnqyP,0.608,0.756,7,-2.675,0,0.176,0.0784,...,audio_features,5BKKy9fIJL5uM9fz1SnqyP,spotify:track:5BKKy9fIJL5uM9fz1SnqyP,https://api.spotify.com/v1/tracks/5BKKy9fIJL5u...,https://api.spotify.com/v1/audio-analysis/5BKK...,245507,4,Pop,g.115,3fMbdgg4jU18AjLCKBhRSm
3,Michael Jackson,Smooth Criminal - 2012 Remaster,spotify:track:2bCQHF9gdG5BNDVuEIEnNk,0.853,0.981,8,-3.947,1,0.0751,0.247,...,audio_features,2bCQHF9gdG5BNDVuEIEnNk,spotify:track:2bCQHF9gdG5BNDVuEIEnNk,https://api.spotify.com/v1/tracks/2bCQHF9gdG5B...,https://api.spotify.com/v1/audio-analysis/2bCQ...,257760,4,Pop,g.115,3fMbdgg4jU18AjLCKBhRSm
4,Michael Jackson,Don't Stop 'Til You Get Enough,spotify:track:46eu3SBuFCXWsPT39Yg3tJ,0.878,0.821,11,-9.875,1,0.089,0.126,...,audio_features,46eu3SBuFCXWsPT39Yg3tJ,spotify:track:46eu3SBuFCXWsPT39Yg3tJ,https://api.spotify.com/v1/tracks/46eu3SBuFCXW...,https://api.spotify.com/v1/audio-analysis/46eu...,365467,4,Pop,g.115,3fMbdgg4jU18AjLCKBhRSm


In [64]:
merged_genre_tracks.to_csv('../Music-Research-Project-1/Resources/top_tracks_genre.csv')